In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.resnest_enhancement as unet
from malaya_speech.train.model import enhancement
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 8192)
model = unet.Model(partitioned_x, channels_interval = 24, n_layers = 13)

down_0 (?, 8192, 24)
Tensor("strided_slice_3:0", shape=(?, 4096, 24), dtype=float32)
down_1 (?, 4096, 48)
Tensor("strided_slice_4:0", shape=(?, 2048, 48), dtype=float32)
down_2 (?, 2048, 72)
Tensor("strided_slice_5:0", shape=(?, 1024, 72), dtype=float32)
down_3 (?, 1024, 96)
Tensor("strided_slice_6:0", shape=(?, 512, 96), dtype=float32)
down_4 (?, 512, 120)
Tensor("strided_slice_7:0", shape=(?, 256, 120), dtype=float32)
down_5 (?, 256, 144)
Tensor("strided_slice_8:0", shape=(?, 128, 144), dtype=float32)
down_6 (?, 128, 168)
Tensor("strided_slice_9:0", shape=(?, 64, 168), dtype=float32)
down_7 (?, 64, 192)
Tensor("strided_slice_10:0", shape=(?, 32, 192), dtype=float32)
down_8 (?, 32, 216)
Tensor("strided_slice_11:0", shape=(?, 16, 216), dtype=float32)
down_9 (?, 16, 240)
Tensor("strided_slice_12:0", shape=(?, 8, 240), dtype=float32)
down_10 (?, 8, 264)
Tensor("strided_slice_13:0", shape=(?, 4, 264), dtype=float32)
down_11 (?, 4, 288)
Tensor("strided_slice_14:0", shape=(?, 2, 288), dtype

In [5]:
model.logits

<tf.Tensor 'sequential_14/activation/Tanh:0' shape=(?, 8192, 1) dtype=float32>

In [6]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_43:0' shape=(?, 1) dtype=float32>

In [7]:
snr = enhancement.loss.snr(model.logits, partitioned_x)
sdr = enhancement.loss.sdr(model.logits, partitioned_x)

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [9]:
import numpy as np

In [10]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [11]:
y_, snr_, sdr_ = sess.run([logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(258867, 1)

In [12]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [13]:
snr_, sdr_

((0.54135877, -19.199749), -2.284933e-07)

In [14]:
y

array([-0.00570545, -0.01051882, -0.01370371, ...,  0.0140948 ,
        0.01236383,  0.01233889])

In [15]:
y_[:,0]

array([-0.68887675, -0.9400899 , -0.95021003, ..., -0.25275633,
       -0.19351546, -0.18663266], dtype=float32)

In [16]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [17]:
!ls -lh test

total 99792
-rw-r--r--  1 huseinzolkepli  staff    77B Feb  8 17:10 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    44M Feb  8 17:10 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    59K Feb  8 17:10 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   5.1M Feb  8 17:10 model.ckpt.meta


In [18]:
!rm -rf test